In [33]:
import pandas as pd
import urllib
from sqlalchemy import create_engine
#
#Connection string for sqlalchemy
connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=QN 1 OLTP;Trusted_Connection=yes;"
#connection_string ="Driver={ODBC Driver 17 for SQL Server};Server=stwssbsql01.ad.okstate.edu;Database=MYStoreDW;Trusted_Connection=yes;"
connection_string = urllib.parse.quote_plus(connection_string)
connection_string = "mssql+pyodbc:///?odbc_connect=%s" % connection_string

In [4]:
df = pd.read_csv('date.csv', parse_dates=['Date'])
print(df.columns)  # This will print the list of columns as read from the CSV


Index(['Date', 'Day_of_Week', 'Year'], dtype='object')


## Dates csv

In [19]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()
class DateData(Base):
    __tablename__ = 'Dates'
    Date = Column(Date, primary_key=True)
    Day_of_Week = Column(String)
    Year = Column(Integer)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path, parse_dates=['Date'])
    with open('task1.txt', 'a') as f:
        f.write("Importing Date csv into Dates table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")
        
    # Check for duplicates
    duplicates = df[df.duplicated(['Date'], keep=False)]  # Checking duplicates based on 'Date'
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 1 : Checking for duplicates. \n")
        if not duplicates.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - DATES : Duplicates found in row no. {index + 1}\n")
        if flagd==0:
            f.write("Table - DATES : No duplicates found.\n")

    # Removing duplicates
    df.drop_duplicates(subset=['Date'], inplace=True)
    
    rows_to_drop = []
    flag2=0
    for index, row in df.iterrows():
        if not isinstance(row['Year'], int):
            flag2=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 2 : Checking for Datatypes. \n")
                out_str = f"Row {index + 1} in Date table {whfile} has a non-numeric Year and will be dropped\n"
                f.write(out_str)
                rows_to_drop.append(index)
    if flag2==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 2 : Checking for Datatypes. \n")
            f.write("Table - DATES : All the entries are of correct datatype.\n")
            #print(f"Row {index + 1} in Date table {whfile} has a non-numeric Year and will be dropped")
            
    df.drop(rows_to_drop, inplace=True)
            
    rows_to_drop1 = []
    flag3=0
    for index, row in df.iterrows():
        if  row['Year']<0:
            flag3=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 3 : Checking for Year range. \n")
                out_str = f"Row {index + 1} in Date table {whfile} has a negative Year (invalid) and will be dropped\n"
                f.write(out_str)
                f.write("\n")
                f.write("\n")
                rows_to_drop1.append(index)
    if flag3==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 3 : Checking for Year range. \n")
            f.write("Table - DATES : Year column entries are in correct range (greater than 0)\n")
            f.write("\n")
            #print(f"Row {index + 1} in Date table {whfile} has a positive Year and will be dropped")
            

    df.drop(rows_to_drop1, inplace=True)

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = DateData(Date=row['Date'], Day_of_Week=row['Day_of_Week'], Year=row['Year'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - DATES : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'date.csv')


C:\Users\phani\AppData\Local\Temp\ipykernel_13504\76810435.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Location csv

In [27]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class DateData(Base):
    __tablename__ = 'Location'
    Latitude = Column(Float,primary_key=True)  # Float is now correctly imported and used
    Longitude = Column(Float,primary_key=True)  # Float is now correctly imported and used
    Location_Easting_OSGR = Column(String)  
    Location_Northing_OSGR = Column(String)  
    LSOA_of_Accident_Location = Column(String)  
    Police_Force = Column(String)  
    Urban_or_Rural_Area = Column(String)  
    InScotland = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing location csv into Location table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")
        
    # Check for duplicates
    duplicates = df[df.duplicated(['Latitude','Longitude'], keep=False)]  # Checking duplicates based on 'Date'
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 1 : Checking for duplicates. \n")
        if not duplicates.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Location : Duplicates found in row no. {index + 1}\n")
        if flagd==0:
            f.write("Table - Location : No duplicates found.\n")

    # Removing duplicates
    df.drop_duplicates(subset=['Latitude','Longitude'], inplace=True)
    
    rows_to_drop = []
    flag2=0
    for index, row in df.iterrows():
        if not isinstance(row['Latitude'], int) & isinstance(row['Longitude'], int):
            flag2=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 2 : Checking for Datatypes. \n")
                out_str = f"Row {index + 1} in Date table  has a non-numeric Latitude& Longitude and will be dropped\n"
                f.write(out_str)
                rows_to_drop.append(index)
    if flag2==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 2 : Checking for Datatypes. \n")
            f.write("Table - Location : All the entries are of correct datatype.\n")
            #print(f"Row {index + 1} in Date table {whfile} has a non-numeric Year and will be dropped")
            
    df.drop(rows_to_drop, inplace=True)

    missing_latitudes = df[(df['Latitude'].isnull())|(df['Longitude'].isnull())]
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 3 : Checking for missing entries. \n")
        if not missing_latitudes.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Location : Missing entry in Primary key at row no: {index + 1}\n")
        if flagd==0:
            f.write("Table - Location : No missing entries.\n")
    #df = df.dropna(subset=['Latitude','Longitude'])


    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = DateData(Latitude=row['Latitude'], Longitude=row['Longitude'], Location_Easting_OSGR=row['Location_Easting_OSGR'],
                              Location_Northing_OSGR=row['Location_Northing_OSGR'], LSOA_of_Accident_Location=row['LSOA_of_Accident_Location'], 
                              Police_Force=row['Police_Force'],Urban_or_Rural_Area=row['Urban_or_Rural_Area'], Day_of_Week=row['InScotland'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Location : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'location.csv')


C:\Users\phani\AppData\Local\Temp\ipykernel_13504\2560385124.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Ped human control CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class LocationData(Base):
    __tablename__ = 'Ped human control'
    Pedestrian_Crossing_Human_Control = Column(Integer,primary_key=True)  # Float is now correctly imported and used
    Meaning = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing Ped human control csv into Pedestrian_Human_Control table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = LocationData(Pedestrian_Crossing_Human_Control=row['Pedestrian_Crossing_Human_Control'],Meaning=row['Meaning'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Pedestrian_Crossing_Human_Control : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'Ped human control.csv')


## Ped Physical faciltiies CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class LocationData(Base):
    __tablename__ = 'Pedestrian_Physical_Facilities'
    Pedestrian_Crossing_Physical_Facilities = Column(Integer,primary_key=True)  # Float is now correctly imported and used
    Meaning = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing Ped Physical faciltiies csv into Pedestrian_Physical_Facilities table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = LocationData(Pedestrian_Crossing_Human_Control=row['Pedestrian_Crossing_Physical_Facilities'],Meaning=row['Meaning'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Pedestrian_Crossing_Physical_Facilities : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'Ped Physical faciltiies.csv')


## Police attended CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class LocationData(Base):
    __tablename__ = 'Police_Attendance'
    Did_Police_Officer_Attend_Scene_of_Accident = Column(Integer,primary_key=True)  # Float is now correctly imported and used
    Meaning = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing Police attended csv into Police_Attendance table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = LocationData(Did_Police_Officer_Attend_Scene_of_Accident=row['Did_Police_Officer_Attend_Scene_of_Accident'],Meaning=row['Meaning'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Police_Attendance : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'Police attended.csv')


## Driver IMD Decile CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class LocationData(Base):
    __tablename__ = 'IMD_Decile'
    Driver_IMD_Decile = Column(Integer,primary_key=True)  # Float is now correctly imported and used
    Meaning = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing Driver IMD Decile csv into IMD_Decile table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = LocationData(Driver_IMD_Decile=row['Driver_IMD_Decile'],Meaning=row['Meaning'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - IMD_Decile : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'Driver IMD Decile.csv')


## Restricted lane CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class LocationData(Base):
    __tablename__ = 'Restricted_Lane'
    Vehicle_Location_Restricted_Lane = Column(Integer,primary_key=True)  # Float is now correctly imported and used
    Meaning = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing Restricted lane csv into Restricted_Lane table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = LocationData(Vehicle_Location_Restricted_Lane=row['Vehicle_Location_Restricted_Lane'],Meaning=row['Meaning'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Restricted_Lane : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'Restricted lane.csv')


## accident_location CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()
class DateData(Base):
    __tablename__ = 'Accident_Location'
    Accident_Index = Column(String, primary_key=True)
    Junction_Location = Column(String)
    Year = Column(Integer)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing accident_location csv into Accident_Location table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")
        
    # Check for duplicates
    duplicates = df[df.duplicated(['Accident_Index'], keep=False)]  # Checking duplicates based on 'Date'
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 1 : Checking for duplicates. \n")
        if not duplicates.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Accident_Location : Duplicates found in row no. {index + 1}\n")
        if flagd==0:
            f.write("Table - Accident_Location : No duplicates found.\n")

    # Removing duplicates
    df.drop_duplicates(subset=['Accident_Index'], inplace=True)
    
    rows_to_drop = []
    flag2=0
    for index, row in df.iterrows():
        if not isinstance(row['Year'], int):
            flag2=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 2 : Checking for Datatypes. \n")
                out_str = f"Row {index + 1} in Accident_Location table {whfile} has a non-numeric Year and will be dropped\n"
                f.write(out_str)
                rows_to_drop.append(index)
    if flag2==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 2 : Checking for Datatypes. \n")
            f.write("Table - Accident_Location : All the entries are of correct datatype.\n")
            #print(f"Row {index + 1} in Date table {whfile} has a non-numeric Year and will be dropped")
            
    df.drop(rows_to_drop, inplace=True)
            
    rows_to_drop1 = []
    flag3=0
    for index, row in df.iterrows():
        if  row['Year']<0:
            flag3=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 3 : Checking for Year range. \n")
                out_str = f"Row {index + 1} in Accident_Location table {whfile} has a negative Year (invalid) and will be dropped\n"
                f.write(out_str)
                f.write("\n")
                f.write("\n")
                rows_to_drop1.append(index)
    if flag3==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 3 : Checking for Year range. \n")
            f.write("Table - Accident_Location : Year column entries are in correct range (greater than 0)\n")
            f.write("\n")
            #print(f"Row {index + 1} in Date table {whfile} has a positive Year and will be dropped")
            

    df.drop(rows_to_drop1, inplace=True)

    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = DateData(Accident_Index=row['Accident_Index'],Junction_Location=row['Junction_Location'], Year=row['Year'])
        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Accident_Location : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'accident_location.csv')


## accidents CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class AccidentData(Base):
    __tablename__ = 'Accidents'
    Accident_Index = Column(String, primary_key=True)
    First_Road_Class = Column(String)
    First_Road_Number = Column(Integer)
    Second_Road_Class = Column(String)
    Second_Road_Number = Column(Integer)
    Accident_Severity = Column(String)
    Carriageway_Hazards = Column(String)
    Date = Column(Date)
    Did_Police_Officer_Attend_Scene_of_Accident = Column(Integer)
    Junction_Control = Column(String)
    Junction_Detail = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Light_Conditions = Column(String)
    Local_District_Authority = Column(String)
    Local_Highway_Authority = Column(String)
    Number_of_Casualities = Column(Integer)
    Number_of_Vehicles = Column(Integer)
    Pedestrian_Crossing_Human_Control = Column(Integer)
    Pedestrian_Crossing_Physical_Facilities = Column(Integer)
    Road_Surface_Conditions = Column(String)
    Road_Type = Column(String)
    Special_Conditions_at_Site = Column(String)
    Speed_limit = Column(Integer)
    Time = Column(Time)
    Weather_Conditions = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing accidents csv into Accidents table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")
        
    # Check for duplicates
    duplicates = df[df.duplicated(['Accident_Index'], keep=False)]  # Checking duplicates based on 'Date'
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 1 : Checking for duplicates. \n")
        if not duplicates.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Accidents : Duplicates found in row no. {index + 1}\n")
        if flagd==0:
            f.write("Table - Accidents : No duplicates found.\n")

    # Removing duplicates
    df.drop_duplicates(subset=['Accident_Index'], inplace=True)
    
    rows_to_drop = []
    flag2=0
    for index, row in df.iterrows():
        if not isinstance(row['Latitude'], int) & isinstance(row['Longitude'], int):
            flag2=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 2 : Checking for Datatypes. \n")
                out_str = f"Row {index + 1} in Accidents table  has a non-numeric Latitude& Longitude and will be dropped\n"
                f.write(out_str)
                rows_to_drop.append(index)
    if flag2==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 2 : Checking for Datatypes. \n")
            f.write("Table - Accidents : All the entries are of correct datatype.\n")
            #print(f"Row {index + 1} in Date table {whfile} has a non-numeric Year and will be dropped")
            
    df.drop(rows_to_drop, inplace=True)

    missing_entry = df[(df['Accident_Index'].isnull())]
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 3 : Checking for missing entries. \n")
        if not missing_entry.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Accidents : Missing entry in Primary key at row no: {index + 1}\n")
        if flagd==0:
            f.write("Table - Accidents : No missing entries.\n")
    #df = df.dropna(subset=['Latitude','Longitude'])


    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = AccidentData(Accident_Index=row['Accident_Index'],First_Road_Class=row['First_Road_Class'],First_Road_Number=row['First_Road_Number'],
        Second_Road_Class=row['Second_Road_Class'],Second_Road_Number=row['Second_Road_Number'],Accident_Severity=row['Accident_Severity'],
        Carriageway_Hazards=row['Carriageway_Hazards'],Date=row['Date'],Did_Police_Officer_Attend_Scene_of_Accident=row['Did_Police_Officer_Attend_Scene_of_Accident'],
        Junction_Control=row['Junction_Control'],Junction_Detail=row['Junction_Detail'],Latitude=row['Latitude'],Longitude=row['Longitude'],
        Light_Conditions=row['Light_Conditions'],Local_District_Authority=row['Local_District_Authority'],Local_Highway_Authority=row['Local_Highway_Authority'],
        Number_of_Casualities=row['Number_of_Casualities'],Number_of_Vehicles=row['Number_of_Vehicles'],Pedestrian_Crossing_Human_Control=row['Pedestrian_Crossing_Human_Control'],
        Pedestrian_Crossing_Physical_Facilities=row['Pedestrian_Crossing_Physical_Facilities'],Road_Surface_Conditions=row['Road_Surface_Conditions'],
        Road_Type=row['Road_Type'],Special_Conditions_at_Site=row['Special_Conditions_at_Site'],Speed_limit=row['Speed_limit'],Time=row['Time'],
        Weather_Conditions=row['Weather_Conditions'])

        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Accidents : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'accidents.csv')


## vehicles CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date  # Include Float here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime

# Define ORM class
Base = declarative_base()

class AccidentData(Base):
    __tablename__ = 'Vehicle'
    Accident_Index = Column(String, primary_key=True)
    Age_Band_of_Driver = Column(String)
    Age_of_Vehicle = Column(Integer)
    Driver_Home_Area_Type = Column(String)
    Driver_IMD_Decile = Column(Integer)
    Engine_Capacity_CC = Column(Integer)
    Hit_Object_in_Carriageway = Column(String)
    Hit_Object_off_Carriageway = Column(String)
    Journey_Purpose_of_Driver = Column(String)
    Make = Column(String)
    Model = Column(String)
    Propulsion_Code = Column(String)
    Sex_of_Driver = Column(String)
    Skidding_and_Overturning = Column(String)
    Towing_and_Articulation = Column(String)
    Vehicle_Leaving_Carriageway = Column(String)
    Vehicle_Location_Restricted_Lane = Column(Integer)
    Vehicle_Manoeuvre = Column(String)
    Vehicle_Reference = Column(Integer, primary_key=True)
    Vehicle_Type = Column(String)
    Was_Vehicle_Left_Hand_Drive = Column(String)
    X1st_Point_of_Impact = Column(String)

def process_csv_data(connection_string, csv_file_path):
    # Engine to read CSV data
    engine_db = create_engine(connection_string)
    
    # Read CSV
    df = pd.read_csv(csv_file_path)
    with open('task1.txt', 'a') as f:
        f.write("Importing vehicles csv into Vehicle table in Database ACCIDENTS")
        f.write("\n")
        f.write("\n")
        
    # Check for duplicates
    duplicates = df[df.duplicated(['Accident_Index','Vehicle_Reference'], keep=False)]  # Checking duplicates based on 'Date'
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 1 : Checking for duplicates. \n")
        if not duplicates.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Vehicle : Duplicates found in row no. {index + 1}\n")
        if flagd==0:
            f.write("Table - Vehicle : No duplicates found.\n")

    # Removing duplicates
    df.drop_duplicates(subset=['Accident_Index','Vehicle_Reference'], inplace=True)
    
    rows_to_drop1 = []
    flag3=0
    for index, row in df.iterrows():
        if  row['Age_of_Vehicle']<0:
            flag3=1
            with open('task1.txt', 'a') as f:
                f.write("Validation check 2 : Checking for Age_of_Vehicle range. \n")
                out_str = f"Row {index + 1} in Vehicle table  has a negative Age_of_Vehicle (invalid) and will be dropped\n"
                f.write(out_str)
                f.write("\n")
                f.write("\n")
                rows_to_drop1.append(index)
    if flag3==0:
        with open('task1.txt', 'a') as f:
            f.write("Validation check 2 : Checking for Age_of_Vehicle range. \n")
            f.write("Table - Vehicle : Age_of_Vehicle column entries are in correct range (greater than 0)\n")
            f.write("\n")

    missing_entry = df[(df['Accident_Index','Vehicle_Reference'].isnull())]
    flagd=0
    # Logging duplicates or lack thereof
    with open('task1.txt', 'a') as f:
        f.write("Validation check 3 : Checking for missing entries. \n")
        if not missing_entry.empty:
            flagd=1
            for index in duplicates.index:
                f.write(f" Table - Vehicle : Missing entry in Primary key at row no: {index + 1}\n")
        if flagd==0:
            f.write("Table - Vehicle : No missing entries.\n")
    #df = df.dropna(subset=['Latitude','Longitude'])


    # Create engine for target database and create table if not exists
    engine_dw = create_engine(connection_string)
    Base.metadata.create_all(engine_dw)

    # Setup session
    Session = sessionmaker(bind=engine_dw)
    session = Session()

    counter = 0
    for index, row in df.iterrows():
        # Create new record and add to the session
        new_record = AccidentData(Accident_Index=row['Accident_Index'],Age_Band_of_Driver=row['Age_Band_of_Driver'],Age_of_Vehicle=row['Age_of_Vehicle'],
        Driver_Home_Area_Type=row['Driver_Home_Area_Type'],Driver_IMD_Decile=row['Driver_IMD_Decile'],Engine_Capacity_CC=row['Engine_Capacity_CC'],
        Hit_Object_in_Carriageway=row['Hit_Object_in_Carriageway'],Hit_Object_off_Carriageway=row['Hit_Object_off_Carriageway'],Journey_Purpose_of_Driver=row['Journey_Purpose_of_Driver'],
        Make=row['Make'],Model=row['Model'],Propulsion_Code=row['Propulsion_Code'],Sex_of_Driver=row['Sex_of_Driver'],Skidding_and_Overturning=row['Skidding_and_Overturning'],
        Towing_and_Articulation=row['Towing_and_Articulation'],Vehicle_Leaving_Carriageway=row['Vehicle_Leaving_Carriageway'],Vehicle_Location_Restricted_Lane=row['Vehicle_Location_Restricted_Lane'],
        Vehicle_Manoeuvre=row['Vehicle_Manoeuvre'],Vehicle_Reference=row['Vehicle_Reference'],Vehicle_Type=row['Vehicle_Type'],
        Was_Vehicle_Left_Hand_Drive=row['Was_Vehicle_Left_Hand_Drive'],X1st_Point_of_Impact=row['X1st_Point_of_Impact'])


        session.add(new_record)
        counter += 1

    session.commit()
    session.close()

    # Logging the number of rows entered
    with open('task1.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"TimeStamp: {dt_str} --- Number of new records loaded IN Table - Vehicle : {counter}\n")
        f.write("-----------------------------------------------------------------------------------------------------------------------------")
        f.write("\n")
        f.write("\n")

process_csv_data(connection_string,'vehicles.csv')
